In [1]:
import re
import datetime
import pandas as pd
import numpy as np
import requests
import io
import json
import random
from main.models import *
from report.models import *
from django.db.models import Sum
from random import randint
from base64 import b64encode, b64decode
from django.contrib.auth.hashers import make_password
from django.core.files.base import ContentFile
from decimal import Decimal
from datetime import datetime, timedelta

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO

In [9]:
# Script to upload Agent data from main.models into report.model.DailySessionllyBusinessllySale
# this uploads only agent data. icustomer data uploders is below

def upload_daily_sale_for_agent_per_date(date, session):
    # loop thru salegroup, sales for given date, given session
    for sg in SaleGroup.objects.filter(date=date, session=session):

        # Find out the union
        print(sg.date)
        union = 'COIMBATORE Union'
        if sg.business.zone.name == 'NILGIRIS':
            union = 'NILGIRIS Union'
        elif sg.business.zone.name == 'TIRUPPUR':
            union = 'TIRUPPUR Union'
        elif sg.business.zone.name == 'CHENNAI Aavin':
            union = 'CHENNAI Aavin'
        else:
            union = 'COIMBATORE Union'

        # get route via route business map
#         route=RouteBusinessMap.objects.filter(business=sg.business, route__session=sg.session)[0].route

        # Create an User table entry for this Agent
        dsbs_inpkts, created =  DailySessionllyBusinessllySale.objects.update_or_create(
            delivery_date=sg.date,
            session=sg.session,
            business=sg.business,
            route=sg.route,
            zone=sg.zone,
            business_type=sg.business.business_type,
            union=union,
            sold_to='Agent',
            defaults = {
                'created_by':User.objects.get(username='kutobot'),
                'modified_by':User.objects.get(username='kutobot'),
                'total_cost':sg.total_cost,
            }

        )
        if created:
            print("\tRow Created: {}\t{}\t{}".format(dsbs_inpkts.delivery_date, dsbs_inpkts.session, dsbs_inpkts.business.code))
        else:
            print("\tRow Exists: {}\t{}\t{}".format(dsbs_inpkts.delivery_date, dsbs_inpkts.session, dsbs_inpkts.business.code))


        # Now loop thru sales for that salegroup
        for sale in sg.sale_set.all():
            print("\t", sale)

            if sale.product.code == 'TM500':
                sale_litre = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.tm500_pkt = sale.count
                dsbs_inpkts.tm500_litre = sale_litre
                dsbs_inpkts.tm_litre += sale_litre
                dsbs_inpkts.milk_litre += sale_litre
                dsbs_inpkts.tm500_cost = sale.cost
                dsbs_inpkts.tm_cost += sale.cost
                dsbs_inpkts.milk_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
                
                
            elif sale.product.code == 'SM250':
                sale_litre = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.std250_pkt = sale.count
                dsbs_inpkts.std250_litre = sale_litre
                dsbs_inpkts.sm_litre += sale_litre
                dsbs_inpkts.milk_litre += sale_litre
                dsbs_inpkts.std250_cost = sale.cost
                dsbs_inpkts.sm_cost += sale.cost
                dsbs_inpkts.milk_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
                
            elif sale.product.code == 'SM500':
                sale_litre = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.std500_pkt = sale.count
                dsbs_inpkts.sm_litre += sale_litre
                dsbs_inpkts.milk_litre += sale_litre
                dsbs_inpkts.std500_litre = sale_litre
                dsbs_inpkts.std500_cost = sale.cost
                dsbs_inpkts.sm_cost += sale.cost
                dsbs_inpkts.milk_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
                
            elif sale.product.code == 'FCM500':
                sale_litre = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.fcm500_pkt = sale.count
                dsbs_inpkts.fcm500_litre = sale_litre
                dsbs_inpkts.fcm_litre += sale_litre
                dsbs_inpkts.milk_litre += sale_litre
                dsbs_inpkts.fcm500_cost = sale.cost
                dsbs_inpkts.fcm_cost += sale.cost
                dsbs_inpkts.milk_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
                
            elif sale.product.code == 'FCM1000':
                sale_litre = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.fcm1000_pkt = sale.count
                dsbs_inpkts.fcm1000_litre = sale_litre
                dsbs_inpkts.fcm_litre += sale_litre
                dsbs_inpkts.milk_litre += sale_litre
                dsbs_inpkts.fcm1000_cost = sale.cost
                dsbs_inpkts.fcm_cost += sale.cost
                dsbs_inpkts.milk_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
                
            elif sale.product.code == 'TNCAN':
                sale_litre = sale.count
                dsbs_inpkts.tmcan = sale.count / 40
                dsbs_inpkts.tmcan_litre = sale_litre
                dsbs_inpkts.tm_litre += sale_litre
                dsbs_inpkts.milk_litre += sale_litre
                dsbs_inpkts.tmcan_cost = sale.cost
                dsbs_inpkts.tm_cost += sale.cost
                dsbs_inpkts.milk_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
            
            elif sale.product.code == 'SMCAN':
                sale_litre = sale.count
                dsbs_inpkts.smcan = sale.count / 40
                dsbs_inpkts.smcan_litre = sale_litre
                dsbs_inpkts.sm_litre += sale_litre
                dsbs_inpkts.milk_litre += sale_litre
                dsbs_inpkts.smcan_cost = sale.cost
                dsbs_inpkts.sm_cost += sale.cost
                dsbs_inpkts.milk_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
                
            elif sale.product.code == 'FCMCAN':
                sale_litre = sale.count
                sbs_inpkts.fcmcan = sale.count / 40
                dsbs_inpkts.fcmcan_litre = sale_litre
                dsbs_inpkts.fcm_litre += sale_litre
                dsbs_inpkts.milk_litre += sale_litre
                dsbs_inpkts.fcmcan_cost = sale.cost
                dsbs_inpkts.fcm_cost += sale.cost
                dsbs_inpkts.milk_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
                
                
            elif sale.product.code == 'CURD500':
                sale_kgs = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.curd500_pkt = sale.count
                dsbs_inpkts.curd500_kgs = sale_kgs
                dsbs_inpkts.curd500_cost = sale.cost
                dsbs_inpkts.curd_kgs += sale_kgs
                dsbs_inpkts.curd_cost += sale.cost
                dsbs_inpkts.total_litre += sale_kgs
                dsbs_inpkts.fermented_products_litre += sale_kgs
                dsbs_inpkts.fermented_products_cost += sale.cost
                
            elif sale.product.code == 'CU150':
                sale_kgs = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.curd150_pkt = sale.count
                dsbs_inpkts.curd150_kgs = sale_kgs
                dsbs_inpkts.curd150_cost = sale.cost
                dsbs_inpkts.curd_kgs += sale_kgs
                dsbs_inpkts.curd_cost += sale.cost
                dsbs_inpkts.total_litre += sale_kgs
                dsbs_inpkts.fermented_products_litre += sale_kgs
                dsbs_inpkts.fermented_products_cost += sale.cost
                
            elif sale.product.code == 'CPCU':
                sale_kgs = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.cupcurd_box = sale.count / 12
                dsbs_inpkts.cupcurd_count = sale.count
                dsbs_inpkts.cupcurd_kgs = sale_kgs
                dsbs_inpkts.cupcurd_cost = sale.cost
                dsbs_inpkts.curd_kgs += sale_kgs
                dsbs_inpkts.curd_cost += sale.cost
                dsbs_inpkts.total_litre += sale_kgs
                dsbs_inpkts.fermented_products_litre += sale_kgs
                dsbs_inpkts.fermented_products_cost += sale.cost
                
            elif sale.product.code == 'CC!KG':
                sale_kgs = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.curd_bucket = sale.count
                dsbs_inpkts.curd_bucket_kgs = sale_kgs
                dsbs_inpkts.curd_bucket_cost = sale.cost
                dsbs_inpkts.curd_kgs += sale_kgs
                dsbs_inpkts.curd_cost += sale.cost
                dsbs_inpkts.total_litre += sale_kgs
                dsbs_inpkts.fermented_products_litre += sale_kgs
                dsbs_inpkts.fermented_products_cost += sale.cost
                
            elif sale.product.code == 'LSI':
                sale_litre = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.lassi200_pkt = sale.count
                dsbs_inpkts.lassi200_kgs = sale_litre
                dsbs_inpkts.lassi200_cost = sale.cost
                dsbs_inpkts.lassi_litre += sale_litre
                dsbs_inpkts.lassi_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
                dsbs_inpkts.fermented_products_litre += sale_litre
                dsbs_inpkts.fermented_products_cost += sale.cost
                
                
            elif sale.product.code == 'BMILK':
                sale_litre = (Decimal(sale.count) * sale.product.quantity) / 1000
                dsbs_inpkts.buttermilk200_pkt = sale.count
                dsbs_inpkts.buttermilk200_litre = sale_litre
                dsbs_inpkts.buttermilk200_cost = sale.cost
                dsbs_inpkts.lassi_litre += sale_litre
                dsbs_inpkts.lassi_cost += sale.cost
                dsbs_inpkts.total_litre += sale_litre
                dsbs_inpkts.fermented_products_litre += sale_litre
                dsbs_inpkts.fermented_products_cost += sale.cost

        dsbs_inpkts.save()


In [10]:
# Script to upload ICustomer data from main.models into report.model.DailySessionllyBusinessllySale
# this uploads only icustomer data. agent data uploders is above

def upload_daily_sale_for_icustomer_per_date(date, session):
    requested_month = datetime.strptime(date, '%Y-%m-%d').month
    requested_year = datetime.strptime(date, '%Y-%m-%d').year
    
    for route in Route.objects.filter(session=session):
        print('Upload started for route: {}'.format(route.name))
        route_business_ids = list(RouteBusinessMap.objects.filter(route=route).values_list('business__id', flat=True))
        for business_id in route_business_ids:
            business = Business.objects.get(id=business_id)
            union = 'COIMBATORE Union'
            if business.zone.name == 'NILGIRIS':
                union = 'NILGIRIS Union'
            elif business.zone.name == 'TIRUPPUR':
                union = 'TIRUPPUR Union'
            elif business.zone.name == 'CHENNAI Aavin':
                union = 'CHENNAI Aavin'
            else:
                union = 'COIMBATORE Union'

            # get all icustomer sale objects for business, for the month, year
            icustomer_sale_obj = ICustomerSale.objects.filter(icustomer_sale_group__business=business, icustomer_sale_group__date__month=requested_month, icustomer_sale_group__date__year=requested_year, icustomer_sale_group__session=session)
            if icustomer_sale_obj.count() > 0:
                dsbs_inpkts, created =  DailySessionllyBusinessllySale.objects.update_or_create(
                    delivery_date=date,
                    session=session,
                    business=business,
                    zone=business.zone,
                    business_type=business.business_type,
                    union=union,
                    route=route,
                    sold_to='ICustomer',
                    defaults = {
                        'created_by':User.objects.get(username='kutobot'),
                        'modified_by':User.objects.get(username='kutobot'),
                        'total_cost' :icustomer_sale_obj.aggregate(Sum('cost'))['cost__sum'],

                    }

                )
                if created:
                    print("\tRow Created: {}\t{}\t{}".format(dsbs_inpkts.delivery_date, dsbs_inpkts.session, dsbs_inpkts.business.code))
                else:
                    print("\tRow Exists: {}\t{}\t{}".format(dsbs_inpkts.delivery_date, dsbs_inpkts.session, dsbs_inpkts.business.code))

                TM500_sale_obj = icustomer_sale_obj.filter(product__code='TM500')
                if not TM500_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.tm500_pkt = TM500_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_litre = (Decimal(dsbs_inpkts.tm500_pkt) * Product.objects.get(code='TM500').quantity) / 1000
                    dsbs_inpkts.tm500_litre = sale_litre
                    dsbs_inpkts.tm500_cost = TM500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.tm_litre += sale_litre
                    dsbs_inpkts.tm_cost += TM500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_cost += TM500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre

                SM250_sale_obj = icustomer_sale_obj.filter(product__code='SM250')
                if not SM250_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.std250_pkt = SM250_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_litre = (Decimal(dsbs_inpkts.std250_pkt) * Product.objects.get(code='SM250').quantity) / 1000
                    dsbs_inpkts.std250_litre = sale_litre
                    dsbs_inpkts.std250_cost = SM250_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.sm_litre += sale_litre
                    dsbs_inpkts.sm_cost += SM250_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_cost += SM250_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre

                SM500_sale_obj = icustomer_sale_obj.filter(product__code='SM500')
                if not SM500_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.std500_pkt = SM500_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_litre = (Decimal(dsbs_inpkts.std500_pkt) * Product.objects.get(code='SM500').quantity) / 1000
                    dsbs_inpkts.std500_litre = sale_litre
                    dsbs_inpkts.std500_cost = SM500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.sm_litre += sale_litre
                    dsbs_inpkts.sm_cost += SM500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_cost += SM500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre

                FCM500_sale_obj = icustomer_sale_obj.filter(product__code='FCM500')
                if not FCM500_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.fcm500_pkt = FCM500_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_litre = (Decimal(dsbs_inpkts.fcm500_pkt) * Product.objects.get(code='FCM500').quantity) / 1000
                    dsbs_inpkts.fcm500_litre = sale_litre
                    dsbs_inpkts.fcm500_cost = FCM500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.fcm_litre += sale_litre
                    dsbs_inpkts.fcm_cost += FCM500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_cost += FCM500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre

                FCM1000_sale_obj = icustomer_sale_obj.filter(product__code='FCM1000')
                if not FCM1000_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.fcm1000_pkt = FCM1000_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_litre = (Decimal(dsbs_inpkts.fcm1000_pkt) * Product.objects.get(code='FCM1000').quantity) / 1000
                    dsbs_inpkts.fcm1000_litre = sale_litre
                    dsbs_inpkts.fcm1000_cost = FCM1000_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.fcm_litre += sale_litre
                    dsbs_inpkts.fcm_cost += FCM1000_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_cost += FCM1000_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre

                TNCAN_sale_obj = icustomer_sale_obj.filter(product__code='TNCAN')
                if not TNCAN_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    sale_litre = TNCAN_sale_obj.aggregate(Sum('count'))['count__sum']
                    dsbs_inpkts.tmcan = TNCAN_sale_obj.aggregate(Sum('count'))['count__sum'] / 40
                    dsbs_inpkts.tmcan_litre = sale_litre
                    dsbs_inpkts.tmcan_cost = TNCAN_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.tm_litre += sale_litre
                    dsbs_inpkts.tm_cost += TNCAN_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_cost += TNCAN_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre

                SMCAN_sale_obj = icustomer_sale_obj.filter(product__code='SMCAN')
                if not SMCAN_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    sale_litre = SMCAN_sale_obj.aggregate(Sum('count'))['count__sum']
                    dsbs_inpkts.smcan = SMCAN_sale_obj.aggregate(Sum('count'))['count__sum'] / 40
                    dsbs_inpkts.smcan_litre = sale_litre
                    dsbs_inpkts.smcan_cost = SMCAN_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.sm_litre += sale_litre
                    dsbs_inpkts.sm_cost += SMCAN_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_cost += SMCAN_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre

                FCMCAN_sale_obj = icustomer_sale_obj.filter(product__code='FCMCAN')
                if not FCMCAN_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    sale_litre = FCMCAN_sale_obj.aggregate(Sum('count'))['count__sum']
                    dsbs_inpkts.fcmcan = FCMCAN_sale_obj.aggregate(Sum('count'))['count__sum'] / 40
                    dsbs_inpkts.fcmcan_litre = sale_litre
                    dsbs_inpkts.fcmcan_cost = FCMCAN_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.fcm_litre += sale_litre
                    dsbs_inpkts.fcm_cost += FCMCAN_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_cost += FCMCAN_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre

                CURD500_sale_obj = icustomer_sale_obj.filter(product__code='CURD500')
                if not CURD500_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.curd500_pkt = CURD500_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_kgs = (Decimal(dsbs_inpkts.curd500_pkt) * Product.objects.get(code='CURD500').quantity) / 1000
                    dsbs_inpkts.curd500_kgs = sale_kgs
                    dsbs_inpkts.curd500_cost = CURD500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.curd_kgs += sale_kgs
                    dsbs_inpkts.curd_cost += CURD500_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkt.total_litre += sale_kgs
                    dsbs_inpkt.fermented_products_litre += sale_kgs
                    dsbs_inpkts.fermented_products_cost += CURD500_sale_obj.aggregate(Sum('cost'))['cost__sum']

                CU150_sale_obj = icustomer_sale_obj.filter(product__code='CU150')
                if not CU150_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.curd150_pkt = CU150_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_kgs = (Decimal(dsbs_inpkts.curd150_pkt) * Product.objects.get(code='CU150').quantity) / 1000
                    dsbs_inpkts.curd150_kgs = sale_kgs
                    dsbs_inpkts.curd150_cost = CU150_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.curd_kgs += sale_kgs
                    dsbs_inpkts.curd_cost += CU150_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.total_litre += sale_kgs
                    dsbs_inpkts.fermented_products_litre += sale_kgs
                    dsbs_inpkts.fermented_products_cost += CU150_sale_obj.aggregate(Sum('cost'))['cost__sum']

                CPCU_sale_obj = icustomer_sale_obj.filter(product__code='CPCU')
                if not CPCU_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.cupcurd = CPCU_sale_obj.aggregate(Sum('count'))['count__sum'] / 12
                    sale_kgs = (Decimal(dsbs_inpkts.cupcurd_count) * Product.objects.get(code='CPCU').quantity) / 1000
                    dsbs_inpkts.cupcurd_count = CPCU_sale_obj.aggregate(Sum('count'))['count__sum']
                    dsbs_inpkts.cupcurd_kgs = sale_kgs
                    dsbs_inpkts.cupcurd_cost = CPCU_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.curd_kgs += sale_kgs
                    dsbs_inpkts.curd_cost += CPCU_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.total_litre += sale_kgs
                    dsbs_inpkts.fermented_products_litre += sale_kgs
                    dsbs_inpkts.fermented_products_cost += CPCU_sale_obj.aggregate(Sum('cost'))['cost__sum']

                CCKG_sale_obj = icustomer_sale_obj.filter(product__code='CC!KG')
                if not CCKG_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.curd_bucket = CCKG_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_kgs = (Decimal(dsbs_inpkts.curd_bucket) * Product.objects.get(code='CC!KG').quantity) / 1000
                    dsbs_inpkts.curd_bucket_kgs = sale_kgs
                    dsbs_inpkts.curd_bucket_cost = CCKG_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.curd_kgs += sale_kgs
                    dsbs_inpkts.curd_cost += CCKG_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.total_litre += sale_kgs
                    dsbs_inpkts.fermented_products_litre += sale_kgs
                    dsbs_inpkts.fermented_products_cost += CCKG_sale_obj.aggregate(Sum('cost'))['cost__sum']

                LSI_sale_obj = icustomer_sale_obj.filter(product__code='LSI')
                if not LSI_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.lassi200_pkt = LSI_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_litre = (Decimal(dsbs_inpkts.lassi200_pkt) * Product.objects.get(code='LSI').quantity) / 1000
                    dsbs_inpkts.lassi200_kgs = sale_litre
                    dsbs_inpkts.lassi200_cost = LSI_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.lassi_litre += sale_litre
                    dsbs_inpkts.lassi_cost += LSI_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.total_litre += sale_litre
                    dsbs_inpkts.fermented_products_litre += sale_litre
                    dsbs_inpkts.fermented_products_cost += LSI_sale_obj.aggregate(Sum('cost'))['cost__sum']

                BMILK_sale_obj = icustomer_sale_obj.filter(product__code='BMILK')
                if not BMILK_sale_obj.aggregate(Sum('count'))['count__sum'] is None:
                    dsbs_inpkts.buttermilk200_pkt = BMILK_sale_obj.aggregate(Sum('count'))['count__sum']
                    sale_litre = (Decimal(dsbs_inpkts.buttermilk200_pkt) * Product.objects.get(code='BMILK').quantity) / 1000
                    dsbs_inpkts.buttermilk200_litre = sale_litre
                    dsbs_inpkts.buttermilk200_cost = BMILK_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    
                    dsbs_inpkts.buttermilk_litre += sale_litre
                    dsbs_inpkts.buttermilk_cost += BMILK_sale_obj.aggregate(Sum('cost'))['cost__sum']
                    dsbs_inpkts.total_litre += sale_litre
                    dsbs_inpkts.fermented_products_litre += sale_litre
                    dsbs_inpkts.fermented_products_cost += BMILK_sale_obj.aggregate(Sum('cost'))['cost__sum']

                dsbs_inpkts.save()
        print('Upload completed for route: {}'.format(route.name))

            

In [16]:
# Script to upload Leakage allowence data from main.models(routetracewisesummary) into report.model.DailySessionllyBusinessllySale
# this uploads only Leakage data. icustomer data and agent data uploders are above

def upload_daily_leakage_allowence_for_each_route_per_session_given_date(date, session):
    print("Processing Leakage allowance. RouteWise")
    # loop thru RouteTrace, RouteTraceSummary for given date, given session
    for rt in RouteTrace.objects.filter(date=date, session=session):
        rtss = rt.routetracewisesalesummary_set.all()
        print("{}\t{}\t{}\t{}".format(rt.route.name, rt.date, rt.session, rtss.count()))





        if rtss.count() > 0:
            # Find out the union
            # Hari how can we not hard code this?
            union = 'COIMBATORE Union'
            if rt.route.name=='NILGIRIS_MOR':
                union = 'NILGIRIS Union'
            elif rt.route.name == 'O - TIRUPP_MOR':
                union = 'TIRUPPUR Union'
            elif rt.route.name == 'O1 - TIRUP_MOR':
                union = 'TIRUPPUR Union'
            elif rt.route.name == 'O2 - TIRUP_MOR':
                union = 'TIRUPPUR Union'
            elif rt.route.name == 'TCMPS-CHENNAI_MOR':
                union = 'CHENNAI Aavin'
            else:
                union = 'COIMBATORE Union'


            # Find out the zone
            guesstimated_zone = RouteBusinessMap.objects.filter(route=rt.route)[0].business.zone


            dsbs_inpkts, created =  DailySessionllyBusinessllySale.objects.update_or_create(
                delivery_date=date,
                session=session,
                # business=business,
                zone=guesstimated_zone,
                # business_type=business.business_type,
                union=union,
                route=rt.route,
                sold_to='Leakage',
                defaults = {
                    'created_by':User.objects.get(username='kutobot'),
                    'modified_by':User.objects.get(username='kutobot'),
                    'total_cost':0,

                }

            )
            if created:
                print("\tRow Created: {}\t{}\t{}".format(dsbs_inpkts.delivery_date, dsbs_inpkts.session, dsbs_inpkts.sold_to))
            else:
                print("\tRow Exists: {}\t{}\t{}".format(dsbs_inpkts.delivery_date, dsbs_inpkts.session, dsbs_inpkts.sold_to))



            for rts in rtss:
                print("\t{},{},{},{}".format(rts.product.code, rts.leak_packet_count, rts.tray_count, rts.loose_packet_count))

                # if none type recieved for leakpacket count set it to zero
                if rts.leak_packet_count == None:
                    rts.leak_packet_count = 0


                if rts.product.code == 'TM500':
                    sale_litre = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.tm500_pkt = rts.leak_packet_count
                    dsbs_inpkts.tm500_litre = sale_litre
                    dsbs_inpkts.tm500_cost = 0
                    
                    dsbs_inpkts.tm_litre += sale_litre
                    dsbs_inpkts.tm_cost += 0
                    dsbs_inpkts.milk_cost += 0
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre
                    
                elif rts.product.code == 'SM250':
                    sale_litre = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.std250_pkt = rts.leak_packet_count
                    dsbs_inpkts.std250_litre = sale_litre
                    dsbs_inpkts.std250_cost = 0
                    
                    dsbs_inpkts.sm_litre += sale_litre
                    dsbs_inpkts.sm_cost += 0
                    dsbs_inpkts.milk_cost += 0
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre
                    
                elif rts.product.code == 'SM500':
                    sale_litre = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.std500_pkt = rts.leak_packet_count
                    dsbs_inpkts.std500_litre = sale_litre
                    dsbs_inpkts.std500_cost = 0
                    
                    dsbs_inpkts.sm_litre += sale_litre
                    dsbs_inpkts.sm_cost += 0
                    dsbs_inpkts.milk_cost += 0
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre
                    
                elif rts.product.code == 'FCM500':
                    sale_litre = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.fcm500_pkt = rts.leak_packet_count
                    dsbs_inpkts.fcm500_litre = sale_litre
                    dsbs_inpkts.fcm500_cost = 0
                    
                    dsbs_inpkts.fcm_litre += sale_litre
                    dsbs_inpkts.fcm_cost += 0
                    dsbs_inpkts.milk_cost += 0
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre
                    
                elif rts.product.code == 'FCM1000':
                    sale_litre = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.fcm1000_pkt = rts.leak_packet_count
                    dsbs_inpkts.fcm1000_litre = sale_litre
                    dsbs_inpkts.fcm1000_cost = 0
                    
                    dsbs_inpkts.fcm_litre += sale_litre
                    dsbs_inpkts.fcm_cost += 0
                    dsbs_inpkts.milk_cost += 0
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre
                    
                elif rts.product.code == 'TNCAN':
                    sale_litre = rts.leak_packet_count
                    dsbs_inpkts.tmcan = rts.leak_packet_count / 40
                    dsbs_inpkts.tmcan_litre = sale_litre
                    dsbs_inpkts.tmcan_cost = 0
                    
                    dsbs_inpkts.tm_litre += sale_litre
                    dsbs_inpkts.tm_cost += 0
                    dsbs_inpkts.milk_cost += 0
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre
                    
                elif rts.product.code == 'SMCAN':
                    sale_litre = rts.leak_packet_count
                    dsbs_inpkts.smcan = rts.leak_packet_count / 40
                    dsbs_inpkts.smcan_litre = sale_litre
                    dsbs_inpkts.smcan_cost = 0
                    
                    dsbs_inpkts.sm_litre += sale_litre
                    dsbs_inpkts.sm_cost += 0
                    dsbs_inpkts.milk_cost += 0
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre
                    
                elif rts.product.code == 'FCMCAN':
                    sale_litre = rts.leak_packet_count
                    dsbs_inpkts.fcmcan = rts.leak_packet_count / 40
                    dsbs_inpkts.fcmcan_litre = sale_litre
                    dsbs_inpkts.fcmcan_cost = 0
                    
                    dsbs_inpkts.fcm_litre += sale_litre
                    dsbs_inpkts.fcm_cost += 0
                    dsbs_inpkts.milk_cost += 0
                    dsbs_inpkts.milk_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre
                    
                elif rts.product.code == 'CURD500':
                    sale_kgs = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.curd500_pkt = rts.leak_packet_count
                    dsbs_inpkts.curd500_kgs = sale_kgs
                    dsbs_inpkts.curd500_cost = 0
                    
                    dsbs_inpkts.curd_kgs += sale_kgs
                    dsbs_inpkts.curd_cost += 0
                    dsbs_inpkts.fermented_products_cost += 0
                    dsbs_inpkts.fermented_products_litre += sale_kgs
                    dsbs_inpkts.total_litre += sale_kgs
                    
                elif rts.product.code == 'CU150':
                    sale_kgs = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.curd150_pkt = rts.leak_packet_count
                    dsbs_inpkts.curd150_kgs = sale_kgs
                    dsbs_inpkts.curd150_cost = 0
                    
                    dsbs_inpkts.curd_kgs += sale_kgs
                    dsbs_inpkts.curd_cost += 0
                    dsbs_inpkts.fermented_products_cost += 0
                    dsbs_inpkts.fermented_products_litre += sale_kgs
                    dsbs_inpkts.total_litre += sale_kgs
                    
                elif rts.product.code == 'CPCU':
                    sale_kgs = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.cupcurd_box = rts.leak_packet_count / 12
                    dsbs_inpkts.cupcurd_count = rts.leak_packet_count
                    dsbs_inpkts.cupcurd_kgs = sale_kgs
                    dsbs_inpkts.cupcurd_cost = 0
                    
                    dsbs_inpkts.curd_kgs += sale_kgs
                    dsbs_inpkts.curd_cost += 0
                    dsbs_inpkts.fermented_products_cost += 0
                    dsbs_inpkts.fermented_products_litre += sale_kgs
                    dsbs_inpkts.total_litre += sale_kgs
                    
                elif rts.product.code == 'CC!KG':
                    sale_kgs = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.curd_bucket = rts.leak_packet_count
                    dsbs_inpkts.curd_bucket_kgs = sale_kgs
                    dsbs_inpkts.curd_bucket_cost = 0
                    
                    dsbs_inpkts.curd_kgs += sale_kgs
                    dsbs_inpkts.curd_cost += 0
                    dsbs_inpkts.fermented_products_cost += 0
                    dsbs_inpkts.fermented_products_litre += sale_kgs
                    dsbs_inpkts.total_litre += sale_kgs
                    
                elif rts.product.code == 'LSI':
                    sale_litre = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.lassi200_pkt = rts.leak_packet_count
                    dsbs_inpkts.lassi200_kgs = sale_litre
                    dsbs_inpkts.lassi200_cost = 0
                    
                    dsbs_inpkts.lassi_litre += sale_litre
                    dsbs_inpkts.lassi_cost += 0
                    dsbs_inpkts.fermented_products_cost += 0
                    dsbs_inpkts.fermented_products_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre
                    
                elif rts.product.code == 'BMILK':
                    sale_litre = (Decimal(rts.leak_packet_count) * rts.product.quantity) / 1000
                    dsbs_inpkts.buttermilk200_pkt = rts.leak_packet_count
                    dsbs_inpkts.buttermilk200_litre = sale_litre
                    dsbs_inpkts.buttermilk200_cost = 0
                    
                    dsbs_inpkts.buttermilk_litre += sale_litre
                    dsbs_inpkts.buttermilk_cost += 0
                    dsbs_inpkts.fermented_products_cost += 0
                    dsbs_inpkts.fermented_products_litre += sale_litre
                    dsbs_inpkts.total_litre += sale_litre

            dsbs_inpkts.save()


In [17]:
datetime.now()

datetime.datetime(2020, 6, 24, 1, 17, 36, 323704)

In [18]:
# Master Script to upload Agent, icustomer data from main.models into report.model.DailySessionllyBusinessllySale
# This simply calls both the functions above for a given date

agent_and_icustomer_scripts = [upload_daily_sale_for_agent_per_date, upload_daily_sale_for_icustomer_per_date, upload_daily_leakage_allowence_for_each_route_per_session_given_date]
#agent_and_icustomer_scripts = [upload_daily_sale_for_icustomer_per_date]
dates = ['2020-06-27']
for date in dates:
    for session in Session.objects.all():
        if DailySessionllyBusinessllySale.objects.filter(delivery_date=date, session=session).exists():
            DailySessionllyBusinessllySale.objects.filter(delivery_date=date, session=session).delete()
            print("Order deleted for date {}".format(date))
        for script in agent_and_icustomer_scripts:
            if SaleGroup.objects.filter(date=date, session=session).exists():
                print("entrie available for {} - {} in sale group. Proceeding to process date {}".format(date, session.name, date))
                script(date, session)
                print("Done processing date {} - {}".format(date, session.name))

            else:
                print("NO entries available for {} - {} in sale group. Skipping {}".format(date, session.name, date))
            


2020-06-20
Order deleted for date 2020-06-20
crossed
entrie available for 2020-06-20 - Evening in sale group. Proceeding to process date 2020-06-20
2020-06-20
	Row Created: 2020-06-20	Eve	3057
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	4187
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	400
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1559
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1758
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1748
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	83
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	58
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	2184
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1623
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	364
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	121
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	432
	 Sale ob

	Row Created: 2020-06-20	Eve	408
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	423
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1149
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	2394
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	316
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1025
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	2439
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	130
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	253
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	217
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	2643
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1637
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	625
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	2679
	 Sale object
	 Sale obj

	Row Created: 2020-06-20	Eve	240
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	296
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	437
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	458
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1507
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1736
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	264
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	2139
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	1538
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	477
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	33
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	2405
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	107
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	381
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Eve	425
	 Sale obje

	Row Created: 2020-06-20	Eve	639
Upload completed for route: 1 - SULUR_EVE
Upload started for route: P - THONDA_EVE
Upload completed for route: P - THONDA_EVE
Upload started for route: 4 - SARAVA_EVE_temp
Upload completed for route: 4 - SARAVA_EVE_temp
Upload started for route: 2 - PERIYA_EVE_temp
Upload completed for route: 2 - PERIYA_EVE_temp
Upload started for route: B1-ANAIKATTY_EVE_temp
Upload completed for route: B1-ANAIKATTY_EVE_temp
Upload started for route: CURD SUPPL_EVE_temp
Upload completed for route: CURD SUPPL_EVE_temp
Upload started for route: V_ROUTE_EVE_temp
Upload completed for route: V_ROUTE_EVE_temp
Upload started for route: CURD SUPPL_EVE
Upload completed for route: CURD SUPPL_EVE
Upload started for route: I - MARUDH_EVE
Upload completed for route: I - MARUDH_EVE
Upload started for route: M - POLLAC_EVE_temp
Upload completed for route: M - POLLAC_EVE_temp
Upload started for route: 2 - PERIYA_EVE
Upload completed for route: 2 - PERIYA_EVE
Upload started for route: O

	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	1821
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	2190
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	872
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	2568
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	14
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	2321
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	2448
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	254
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	4190
	 Sale object
2020-06-20
	Row Created: 2020-06-20	Mor	343
	 Sale object
	 Sale ob

...

	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	1526
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	2444
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	2197
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	9029
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	532
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	137
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	477
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	1538
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	469
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	74
	 Sale object
	 Sale object
	 Sale object
2020-06-21

	Row Created: 2020-06-21	Mor	2342
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	2365
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	2438
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	2566
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	128
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	25
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	2643
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	2621
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	602
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	395
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
2020-06-21
	Row Created: 2020-06-21	Mor	2391
	 Sale object
	 Sale object
	 Sale object
	 Sale object
	 Sale object
Don

	Row Created: 2020-06-21	Mor	58
	Row Created: 2020-06-21	Mor	711
	Row Created: 2020-06-21	Mor	57
	Row Created: 2020-06-21	Mor	1597
	Row Created: 2020-06-21	Mor	2218
	Row Created: 2020-06-21	Mor	328
	Row Created: 2020-06-21	Mor	1532
	Row Created: 2020-06-21	Mor	146
	Row Created: 2020-06-21	Mor	1043
	Row Created: 2020-06-21	Mor	55
Upload completed for route: D - MTP RO_MOR
Upload started for route: E - TOWN_MOR
	Row Created: 2020-06-21	Mor	18
	Row Created: 2020-06-21	Mor	44
	Row Created: 2020-06-21	Mor	88
	Row Created: 2020-06-21	Mor	123
	Row Created: 2020-06-21	Mor	137
	Row Created: 2020-06-21	Mor	142
	Row Created: 2020-06-21	Mor	143
	Row Created: 2020-06-21	Mor	149
	Row Created: 2020-06-21	Mor	203
	Row Created: 2020-06-21	Mor	2247
	Row Created: 2020-06-21	Mor	132
	Row Created: 2020-06-21	Mor	2596
Upload completed for route: E - TOWN_MOR
Upload started for route: O2 - TIRUP_MOR
Upload completed for route: O2 - TIRUP_MOR
Upload started for route: O1 - TIRUP_MOR
Upload completed for route

	Row Created: 2020-06-21	Mor	1727
	Row Created: 2020-06-21	Mor	1526
	Row Created: 2020-06-21	Mor	406
	Row Created: 2020-06-21	Mor	1786
	Row Created: 2020-06-21	Mor	740
Upload completed for route: X - Route_MOR
Upload started for route: J - RAMANA_MOR
	Row Created: 2020-06-21	Mor	83
	Row Created: 2020-06-21	Mor	112
	Row Created: 2020-06-21	Mor	156
	Row Created: 2020-06-21	Mor	437
	Row Created: 2020-06-21	Mor	447
	Row Created: 2020-06-21	Mor	458
	Row Created: 2020-06-21	Mor	481
	Row Created: 2020-06-21	Mor	514
	Row Created: 2020-06-21	Mor	92
	Row Created: 2020-06-21	Mor	13
	Row Created: 2020-06-21	Mor	276
	Row Created: 2020-06-21	Mor	1082
	Row Created: 2020-06-21	Mor	1892
	Row Created: 2020-06-21	Mor	2211
	Row Created: 2020-06-21	Mor	2321
	Row Created: 2020-06-21	Mor	470
	Row Created: 2020-06-21	Mor	14
	Row Created: 2020-06-21	Mor	419
	Row Created: 2020-06-21	Mor	1883
	Row Created: 2020-06-21	Mor	417
	Row Created: 2020-06-21	Mor	418
	Row Created: 2020-06-21	Mor	2656
	Row Created: 2020-06

	Row Created: 2020-06-21	Mor	Leakage
	SM250,0,8,16
	SM500,0,9,4
	FCM500,0,27,0
K - METTUP_MOR	2020-06-21	Mor	6
	Row Created: 2020-06-21	Mor	Leakage
	FCM1000,0,3,0
	SM250,0,14,0
	SM500,0,28,4
	FCM500,0,11,2
	CURD500,0,4,0
	CU150,0,7,0
K1 - METTU_MOR	2020-06-21	Mor	7
	Row Created: 2020-06-21	Mor	Leakage
	SM500,0,77,10
	FCM500,0,217,10
	FCM1000,0,0,5
	SM250,0,101,8
	BMILK,0,0,5
	CURD500,0,9,0
	CU150,0,20,11
A - PODANU_MOR_temp	2020-06-21	Mor	0
O - TIRUPP_MOR	2020-06-21	Mor	11
	Row Created: 2020-06-21	Mor	Leakage
	BMILK,None,None,None
	FCMCAN,None,None,None
	SMCAN,None,None,None
	TM500,None,None,None
	SM500,3,81,0
	CU150,0,12,15
	FCM500,11,231,4
	CPCU,None,None,None
	CURD500,0,3,0
	FCM1000,0,4,6
	SM250,9,96,4
O1 - TIRUP_MOR	2020-06-21	Mor	6
	Row Created: 2020-06-21	Mor	Leakage
	SM250,7,83,0
	SM500,5,110,0
	FCM500,6,127,0
	CURD500,0,4,12
	CU150,0,13,0
	FCM1000,0,13,2
O2 - TIRUP_MOR	2020-06-21	Mor	6
	Row Created: 2020-06-21	Mor	Leakage
	CURD500,0,6,0
	CU150,0,17,15
	FCM1000,0,26,0
	SM250,12,

In [8]:
datetime.now()

datetime.datetime(2020, 5, 18, 3, 0, 5, 879058)

In [9]:
DailySessionllyRoutellySale.objects.filter(delivery_date='2020-05-18').delete()

(157, {'report.DailySessionllyRoutellySale': 157})

In [5]:
for rowobj in DailySessionllyBusinessllySale.objects.all():
    business = rowobj.business
    print(business.zone.name)
    print(business.business_type)
    rowobj.zone = business.zone
    rowobj.business_type = business.business_type
    if business.zone.name == 'NILGIRIS Union':
        rowobj.union = 'NILGIRIS'
    elif business.zone.name == 'TIRUPPUR':
        rowobj.union = 'TIRUPPUR Union'
    elif business.zone.name == 'CHENNAI Aavin':
        rowobj.union = 'CHENNAI Aavin'
    else:
        rowobj.union = 'COIMBATORE Union'
    rowobj.save()
        

MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
EAST
1 - Booth
EAST
1 - Booth
NORTH
1 - Booth
MTP
1 - Booth
NORTH
1 - Booth
MTP
1 - Booth
NORTH
1 - Booth
MTP
1 - Booth
NORTH
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
EAST
1 - Booth
MTP
1 - Booth
NORTH
1 - Booth
MTP
1 - Booth
NORTH
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
EAST
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
EAST
1 - Booth
MTP
1 - Booth
SOUTH
1 - Booth
MTP
1 - Booth
SOUTH
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
SOUTH
2 - Parlour
WEST
1 - Booth
NORTH
1 - Booth
WEST
1 - Booth
SOUTH
1 - Booth
NORTH
1 - Booth
WEST
1 

WEST
1 - Booth
WEST
1 - Booth
SOUTH
1 - Booth
NORTH
1 - Booth
SOUTH
1 - Booth
EAST
1 - Booth
SOUTH
1 - Booth
EAST
1 - Booth
WEST
1 - Booth
EAST
2 - Parlour
EAST
1 - Booth
WEST
1 - Booth
EAST
1 - Booth
EAST
1 - Booth
CURD Zone
9 - Whole Sale Dealer
WEST
1 - Booth
EAST
1 - Booth
WEST
1 - Booth
WEST
5 - Govt Institute
NORTH
1 - Booth
WEST
5 - Govt Institute
WEST
5 - Govt Institute
EAST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
EAST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
3 - Own parlour
WEST
2 - Parlour
EAST
1 - Booth
EAST
1 - Booth
WEST
9 - Whole Sale Dealer
WEST
1 - Booth
SOUTH
2 - Parlour
SOUTH
9 - Whole Sale Dealer
WEST
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
EAST
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
2 - Parlour
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
NORTH
5 - Govt Institute
NORTH
5 - Govt Institute
NORTH
3 - Own parlour
SOUTH
9 - Whole Sale Dealer
NO

SOUTH
1 - Booth
SOUTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
WEST
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
MTP
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
SOUTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
NORTH
1 - Booth
WES

In [20]:
for session in ['Morning', 'Evening']:
        for route in Route.objects.filter(session=Session.objects.get(name=session)):
            route_business_ids = list(RouteBusinessMap.objects.filter(route=route).values_list('business__id', flat=True))
            for business_id in route_business_ids:
                print("----------------")
                print(business_id)
                print("----------------")
                business = Business.objects.get(id=business_id)
                union = 'COIMBATORE Union'
                if business.zone.name == 'NILGIRIS':
                    union = 'NILGIRIS Union'
                elif business.zone.name == 'TIRUPPUR':
                    union = 'TIRUPPUR Union'
                elif business.zone.name == 'CHENNAI Aavin':
                    union = 'CHENNAI Aavin'
                else:
                    union = 'COIMBATORE Union'
                print(union)

    
       

----------------
1298
----------------
COIMBATORE Union
----------------
1304
----------------
COIMBATORE Union
----------------
1320
----------------
COIMBATORE Union
----------------
1324
----------------
COIMBATORE Union
----------------
1325
----------------
COIMBATORE Union
----------------
1326
----------------
COIMBATORE Union
----------------
1333
----------------
COIMBATORE Union
----------------
1341
----------------
COIMBATORE Union
----------------
1346
----------------
COIMBATORE Union
----------------
1311
----------------
COIMBATORE Union
----------------
1292
----------------
COIMBATORE Union
----------------
1314
----------------
COIMBATORE Union
----------------
1353
----------------
COIMBATORE Union
----------------
1358
----------------
COIMBATORE Union
----------------
1359
----------------
COIMBATORE Union
----------------
1365
----------------
COIMBATORE Union
----------------
1373
----------------
COIMBATORE Union
----------------
1381
----------------
COIMBATOR

COIMBATORE Union
----------------
1601
----------------
COIMBATORE Union
----------------
1603
----------------
COIMBATORE Union
----------------
1597
----------------
COIMBATORE Union
----------------
1598
----------------
COIMBATORE Union
----------------
1605
----------------
COIMBATORE Union
----------------
1610
----------------
COIMBATORE Union
----------------
1614
----------------
COIMBATORE Union
----------------
1616
----------------
COIMBATORE Union
----------------
1617
----------------
COIMBATORE Union
----------------
1629
----------------
COIMBATORE Union
----------------
1633
----------------
COIMBATORE Union
----------------
1644
----------------
COIMBATORE Union
----------------
1646
----------------
COIMBATORE Union
----------------
1650
----------------
COIMBATORE Union
----------------
1658
----------------
COIMBATORE Union
----------------
1705
----------------
COIMBATORE Union
----------------
1706
----------------
COIMBATORE Union
----------------
1708
---------

COIMBATORE Union
----------------
2284
----------------
COIMBATORE Union
----------------
2300
----------------
COIMBATORE Union
----------------
2301
----------------
COIMBATORE Union
----------------
2303
----------------
COIMBATORE Union
----------------
2309
----------------
COIMBATORE Union
----------------
2312
----------------
COIMBATORE Union
----------------
2313
----------------
COIMBATORE Union
----------------
2277
----------------
COIMBATORE Union
----------------
2332
----------------
COIMBATORE Union
----------------
2340
----------------
COIMBATORE Union
----------------
2359
----------------
COIMBATORE Union
----------------
2385
----------------
COIMBATORE Union
----------------
2410
----------------
COIMBATORE Union
----------------
2237
----------------
COIMBATORE Union
----------------
2329
----------------
COIMBATORE Union
----------------
2422
----------------
COIMBATORE Union
----------------
2208
----------------
COIMBATORE Union
----------------
2648
---------

COIMBATORE Union
----------------
1695
----------------
COIMBATORE Union
----------------
1677
----------------
COIMBATORE Union
----------------
1680
----------------
COIMBATORE Union
----------------
2577
----------------
COIMBATORE Union
----------------
1674
----------------
COIMBATORE Union
----------------
2655
----------------
COIMBATORE Union
----------------
2537
----------------
TIRUPPUR Union
----------------
2539
----------------
TIRUPPUR Union
----------------
2540
----------------
TIRUPPUR Union
----------------
2545
----------------
TIRUPPUR Union
----------------
2547
----------------
TIRUPPUR Union
----------------
2548
----------------
TIRUPPUR Union
----------------
2551
----------------
TIRUPPUR Union
----------------
2555
----------------
TIRUPPUR Union
----------------
2557
----------------
TIRUPPUR Union
----------------
2546
----------------
TIRUPPUR Union
----------------
2554
----------------
TIRUPPUR Union
----------------
2541
----------------
TIRUPPUR Union

COIMBATORE Union
----------------
1563
----------------
COIMBATORE Union
----------------
1532
----------------
COIMBATORE Union
----------------
1511
----------------
COIMBATORE Union
----------------
1338
----------------
COIMBATORE Union
----------------
2633
----------------
COIMBATORE Union
----------------
1390
----------------
COIMBATORE Union
----------------
1397
----------------
COIMBATORE Union
----------------
1440
----------------
COIMBATORE Union
----------------
1487
----------------
COIMBATORE Union
----------------
2161
----------------
COIMBATORE Union
----------------
1778
----------------
COIMBATORE Union
----------------
1787
----------------
COIMBATORE Union
----------------
1793
----------------
COIMBATORE Union
----------------
1804
----------------
COIMBATORE Union
----------------
1807
----------------
COIMBATORE Union
----------------
1810
----------------
COIMBATORE Union
----------------
1811
----------------
COIMBATORE Union
----------------
1812
---------

...

TIRUPPUR Union
----------------
2553
----------------
TIRUPPUR Union
----------------
1293
----------------
COIMBATORE Union
----------------
1301
----------------
COIMBATORE Union
----------------
1321
----------------
COIMBATORE Union
----------------
1327
----------------
COIMBATORE Union
----------------
1302
----------------
COIMBATORE Union
----------------
1313
----------------
COIMBATORE Union
----------------
1369
----------------
COIMBATORE Union
----------------
1347
----------------
COIMBATORE Union
----------------
1372
----------------
COIMBATORE Union
----------------
1376
----------------
COIMBATORE Union
----------------
1377
----------------
COIMBATORE Union
----------------
1379
----------------
COIMBATORE Union
----------------
1380
----------------
COIMBATORE Union
----------------
1384
----------------
COIMBATORE Union
----------------
1391
----------------
COIMBATORE Union
----------------
1394
----------------
COIMBATORE Union
----------------
1400
-------------

COIMBATORE Union
----------------
1790
----------------
COIMBATORE Union
----------------
1792
----------------
COIMBATORE Union
----------------
1795
----------------
COIMBATORE Union
----------------
1801
----------------
COIMBATORE Union
----------------
1841
----------------
COIMBATORE Union
----------------
1863
----------------
COIMBATORE Union
----------------
1866
----------------
COIMBATORE Union
----------------
1868
----------------
COIMBATORE Union
----------------
1880
----------------
COIMBATORE Union
----------------
1885
----------------
COIMBATORE Union
----------------
1907
----------------
COIMBATORE Union
----------------
1930
----------------
COIMBATORE Union
----------------
1984
----------------
COIMBATORE Union
----------------
2024
----------------
COIMBATORE Union
----------------
2032
----------------
COIMBATORE Union
----------------
2581
----------------
COIMBATORE Union
----------------
1865
----------------
COIMBATORE Union
----------------
1948
---------

COIMBATORE Union
----------------
2126
----------------
COIMBATORE Union
----------------
2127
----------------
COIMBATORE Union
----------------
2129
----------------
COIMBATORE Union
----------------
2131
----------------
COIMBATORE Union
----------------
2135
----------------
COIMBATORE Union
----------------
2136
----------------
COIMBATORE Union
----------------
2141
----------------
COIMBATORE Union
----------------
2142
----------------
COIMBATORE Union
----------------
2143
----------------
COIMBATORE Union
----------------
2148
----------------
COIMBATORE Union
----------------
2149
----------------
COIMBATORE Union
----------------
2150
----------------
COIMBATORE Union
----------------
2152
----------------
COIMBATORE Union
----------------
2155
----------------
COIMBATORE Union
----------------
2111
----------------
COIMBATORE Union
----------------
2128
----------------
COIMBATORE Union
----------------
2132
----------------
COIMBATORE Union
----------------
2137
---------

COIMBATORE Union
----------------
1673
----------------
COIMBATORE Union
----------------
1685
----------------
COIMBATORE Union
----------------
1691
----------------
COIMBATORE Union
----------------
1693
----------------
COIMBATORE Union
----------------
1702
----------------
COIMBATORE Union
----------------
1707
----------------
COIMBATORE Union
----------------
1710
----------------
COIMBATORE Union
----------------
1713
----------------
COIMBATORE Union
----------------
1714
----------------
COIMBATORE Union
----------------
1715
----------------
COIMBATORE Union
----------------
1743
----------------
COIMBATORE Union
----------------
1747
----------------
COIMBATORE Union
----------------
1748
----------------
COIMBATORE Union
----------------
1751
----------------
COIMBATORE Union
----------------
1763
----------------
COIMBATORE Union
----------------
1766
----------------
COIMBATORE Union
----------------
1767
----------------
COIMBATORE Union
----------------
1768
---------

COIMBATORE Union
----------------
2382
----------------
COIMBATORE Union
----------------
2572
----------------
COIMBATORE Union
----------------
2200
----------------
COIMBATORE Union
----------------
2249
----------------
COIMBATORE Union
----------------
2253
----------------
COIMBATORE Union
----------------
2263
----------------
COIMBATORE Union
----------------
2315
----------------
COIMBATORE Union
----------------
2328
----------------
COIMBATORE Union
----------------
2377
----------------
COIMBATORE Union
----------------
2426
----------------
COIMBATORE Union
----------------
2296
----------------
COIMBATORE Union
----------------
2344
----------------
COIMBATORE Union
----------------
2428
----------------
COIMBATORE Union
----------------
1727
----------------
COIMBATORE Union
----------------
1823
----------------
COIMBATORE Union
----------------
1832
----------------
COIMBATORE Union
----------------
1833
----------------
COIMBATORE Union
----------------
1836
---------

In [15]:
for session in ['Morning', 'Evening']:
        for route in Route.objects.filter(session=Session.objects.get(name=session)):
            route_business_codes = list(RouteBusinessMap.objects.filter(route=route).values_list('business', flat=True))
            for business_code in route_business_codes:
                print("----------------")
                print(business_code)
                print("----------------")
                business = Business.objects.get(code=business_code)
                union = 'COIMBATORE Union'
                if business.zone.name == 'NILGIRIS':
                    union = 'NILGIRIS Union'
                elif business.zone.name == 'TIRUPPUR':
                    union = 'TIRUPPUR Union'
                elif business.zone.name == 'CHENNAI Aavin':
                    union = 'CHENNAI Aavin'
                else:
                    union = 'COIMBATORE Union'

<Business: 1714>

In [21]:
DailySessionllyZonallySale.objects.all().delete()

(24, {'report.DailySessionllyZonallySale': 24})

In [19]:
RouteBusinessMap.objects.filter(route=route).values_list('business__id', flat=True)

<QuerySet [1298, 1304, 1320, 1324, 1325, 1326, 1333, 1341, 1346, 1311, 1292, 1314, 1353, 1358, 1359, 1365, 1373, 1381, 1348, 1349, '...(remaining elements truncated)...']>